In [28]:
import requests
import datetime
import time
import tqdm
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
df = pd.DataFrame(columns=['title', 'date', 'contentLink', 'content'])

In [32]:
def getArticles(parsedHTML):
    articles = parsedHTML.find_all('article', class_='card')

    for article in articles:
        title = article.find('h2', class_='card-title').text.strip()
        date = article.find('span', class_='card-post-date').text.strip()
        contentLink = article.find("a", class_="stretched-link")['href']

        ArticlePage = requests.get(contentLink).text 
        parsedArticle = BeautifulSoup(ArticlePage, 'html.parser')
        content = parsedArticle.find('div', class_='post-content').find_all('p')
        df.loc[len(df)] = [title, date, contentLink, '\n'.join([p.text for p in content if "<img" not in str(p) and "<iframe" not in str(p) and "<a" not in str(p)])]


In [33]:
def scrapeArticles():
    end_date = datetime.date(2020, 1, 1)
    current_date = datetime.date.today()

    total_days = (current_date - end_date).days + 1

    for i in tqdm.tqdm(range(0, total_days, 2), desc="Scraping by date"):
        day = current_date - datetime.timedelta(days=i)
        dt = datetime.datetime.combine(day, datetime.time())
        timestamp = int(time.mktime(dt.timetuple()))

        url = f"https://www.goud.ma/wp-json/?load_more=1&after={timestamp}&taxonomy=category&term=3"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            getArticles(soup)
        else:
            print(f"Failed on {day} (Status {response.status_code})")

In [34]:
scrapeArticles()

Scraping by date: 100%|██████████| 1018/1018 [4:54:43<00:00, 17.37s/it] 


In [ ]:
df.to_csv('data/raw/articles.csv')